In [10]:
import requests
import json
import re
import pandas as pd
from bat import bro_log_reader
from bat.log_to_dataframe import LogToDataFrame

In [11]:
#Connect to Bro Logs
bro_df = LogToDataFrame('/usr/local/bro/logs/current/conn.log')
df1=bro_df[['id.orig_h','id.resp_h','id.resp_p','service']]
df1.head(10)
#df1

Successfully monitoring /usr/local/bro/logs/current/conn.log...


,id.orig_h,id.resp_h,id.resp_p,service
ts,,,,
2018-09-25 08:00:00.424851,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.431412,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.437441,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.454782,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.458257,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.463125,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.467550,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.473187,192.168.1.168,192.168.1.1,53,dns
2018-09-25 08:00:00.479340,192.168.1.168,192.168.1.1,53,dns


In [6]:
#Create Threat Intel DataFrame
import urllib3
#######################################################################################################################
########################################## Firehol Intel ##############################################################
#######################################################################################################################
#threatIntel = 'https://iplists.firehol.org/files/tor_exits_30d.ipset'             #Tor Exit Nodes Last 30 Days
#threatIntel = 'https://iplists.firehol.org/files/cidr_report_bogons.netset'
threatIntel = 'https://iplists.firehol.org/files/bm_tor.ipset'
#threatIntel = 'https://iplists.firehol.org/files/blocklist_net_ua.ipset'
#threatIntel = 'https://iplists.firehol.org/files/firehol_abusers_1d.netset'

http = urllib3.PoolManager()
response = http.request('GET', threatIntel)

In [9]:
# Parse Malicious IP Addresses out of the Firehol Feed
rawlist = response.data.splitlines()
list = filter(lambda x: x.split('#')[0], rawlist)
threat_df = pd.DataFrame(list, columns = ["Address"])
#threat_df.head(10)

In [8]:
joined_ips=pd.merge(threat_df,df1, left_on='Address', right_on='id.resp_h')
#joined_ips=pd.merge(ssh_df,threat_df, on='Address', how='inner')
#joined_ips.to_csv('attack.csv')
joined_ips.to_html('CTI_Data_BroConnLog.html')
joined_ips


,Address,id.orig_h,id.resp_h,id.resp_p,service
0,51.15.75.120,192.168.1.168,51.15.75.120,443,-
1,148.251.221.163,192.168.1.168,148.251.221.163,9001,-
2,193.234.15.62,192.168.1.168,193.234.15.62,443,-
